In [1]:
import os
import sys

import numpy as np
from tqdm import trange
from astropy.io import fits
from astropy.table import Table, vstack
from astropy.convolution import convolve, Gaussian1DKernel
import astropy.units as u
import astropy.coordinates as coord
import matplotlib
import matplotlib.pyplot as plt
from astropy.table import Column
from tqdm import trange
import pandas as pd
import fitsio
from astropy.table import Table, vstack
from astropy import units as u
from astropy.coordinates import SkyCoord
from easyquery import Query, QueryMaker
from scipy.stats import binomtest
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import LogNorm
from matplotlib.colors import ListedColormap, BoundaryNorm
import h5py
from astropy.cosmology import Planck18

mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['axes.xmargin'] = 1
mpl.rcParams['xtick.labelsize'] = 'x-large'
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['ytick.labelsize'] = 'x-large'
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['legend.frameon'] = False

rootdir = '/global/u1/v/virajvm/'
sys.path.append(os.path.join(rootdir, 'DESI2_LOWZ/desi_dwarfs/code'))

from desi_lowz_funcs import make_subplots

In [8]:
# DESI targeting masks - 
from desitarget.sv1 import sv1_targetmask    # For SV1
from desitarget.sv2 import sv2_targetmask    # For SV2
from desitarget.sv3 import sv3_targetmask    # For SV3
from desitarget import targetmask



In [2]:
zpix_iron = Table.read("/global/cfs/cdirs/desi/spectro/redux/iron/zcatalog/v1/zall-pix-iron.fits")

#we remove these columns as we will be adding columns from the zpix_trac later and so hope to avoid confusion
cols_to_remove = [
"FLUX_G", "FLUX_R", "FLUX_Z", "FLUX_W1", "FLUX_W2",
"FLUX_IVAR_G", "FLUX_IVAR_R", "FLUX_IVAR_Z", "FLUX_IVAR_W1", "FLUX_IVAR_W2",
"FIBERFLUX_G", "FIBERFLUX_Z",
"FIBERTOTFLUX_G", "FIBERTOTFLUX_R", "FIBERTOTFLUX_Z",
"SERSIC", "SHAPE_R", "SHAPE_E1", "SHAPE_E2"]


zpix_iron.remove_columns(cols_to_remove)



##rename columns
zpix_iron.rename_column('TARGET_RA', 'RA')
zpix_iron.rename_column('TARGET_DEC', 'DEC')


In [3]:

##apply a basic cut of spectype = "GALAXY", z > 0.001 and ZCAT_PRIMARY = 1, and maximum redshift
basic_cleaning = (zpix_iron["SPECTYPE"] == "GALAXY") & (zpix_iron["ZCAT_PRIMARY"] == 1) & (zpix_iron["COADD_FIBERSTATUS"] == 0) & (zpix_iron["Z"] < 0.5) & (zpix_iron["Z"] > 0.001)

#we apply a cut on redshift, because we are using redshifts to get distances and we cannot get accurate stellar masses anyways if z<0.001

print(f"Fraction remaining after basic cleaning cut = { np.sum(basic_cleaning)/len(zpix_iron) }")

zpix_iron = zpix_iron[basic_cleaning]

#select for unique targets
_, uni_tgids = np.unique( zpix_iron["TARGETID"].data, return_index=True )
zpix_iron = zpix_iron[uni_tgids]



Fraction remaining after basic cleaning cut = 0.27931008001382396


In [ ]:

gal_types = ["BGS_BRIGHT", "BGS_FAINT", "ELG","LOWZ"]


#looping over all the sub-samples!
#gal_mask is the boolean array to select the galaxies
for i,gal_type in enumerate(gal_types):

    if gal_type == "LOWZ":
        zpix_iron = zpix_iron[zpix_iron["PROGRAM"] == "dark"]

        #get the lowz survey mask
        lowz_main_mask = (zpix_iron["SCND_TARGET"] == 2**15) | (zpix_iron["SCND_TARGET"] == 2**16) | (zpix_iron["SCND_TARGET"] == 2**17)
        lowz_sv_mask = np.zeros(len(zpix)).astype(bool)
        for svi in range(1,4):
            svi_mask = (zpix[f"SV{svi}_SCND_TARGET"] == 2**15) | (zpix[f"SV{svi}_SCND_TARGET"] == 2**16) | (zpix[f"SV{svi}_SCND_TARGET"] == 2**17)
            lowz_sv_mask |= svi_mask
    
        gal_mask = lowz_main_mask | lowz_sv_mask
            
        
    if gal_type == "BGS_BRIGHT":
        iron_bgs_tgid = zpix_iron["BGS_TARGET"]
        bgsb_main_mask = ( (iron_bgs_tgid & bgs_mask["BGS_BRIGHT"]) != 0 )
        bgsb_sv_mask = get_sv_bgs_mask(zpix_iron, bgs_class = "BGS_BRIGHT")

        print(f"Number of BGS Bright objects in Main survey = {np.sum(bgsb_main_mask)}")
        print(f"Number of BGS Bright objects in SV survey = {np.sum(bgsb_sv_mask)}")
        
        gal_mask = bgsb_main_mask | bgsb_sv_mask
        
    if gal_type == "BGS_FAINT":
        iron_bgs_tgid = zpix_iron["BGS_TARGET"]
        
        bgsf_main_mask = ( (iron_bgs_tgid & bgs_mask["BGS_FAINT"]) != 0 )

        bgsf_sv_mask = get_sv_bgs_mask(zpix_iron, bgs_class = "BGS_FAINT")

        print(f"Number of BGS Faint objects in Main survey = {np.sum(bgsf_main_mask)}")
        print(f"Number of BGS Faint objects in SV survey = {np.sum(bgsf_sv_mask)}")
        
        gal_mask = bgsf_main_mask | bgsf_sv_mask

    if gal_type == "ELG":
        desi_tgt = zpix_iron["DESI_TARGET"]
        elg_main_mask = (desi_tgt & desi_mask["ELG"]) != 0
        elg_sv_mask = get_sv_elg_mask(zpix_iron)
        gal_mask = elg_main_mask | elg_sv_mask




The weird issue of trying to select with fastspec catalog

In [9]:
iron_vac = fits.open("/global/cfs/cdirs/desi/public/dr1/vac/dr1/fastspecfit/iron/v2.1/catalogs/fastspec-iron.fits")
vac_data_f = iron_vac[2].data


In [10]:
## METHOD 1

sel = (vac_data_f["SV1_BGS_TARGET"] & sv1_targetmask.bgs_mask["BGS_BRIGHT"] != 0)

print(len(vac_data_f[sel]))

16692760


In [11]:
## METHOD 2

sv1_desi_tgt =vac_data_f['SV1_DESI_TARGET']
## first select for SV1 BGS ANY objects
sv1_desi_mask = sv1_targetmask.desi_mask
sv1_bgs = (sv1_desi_tgt & sv1_desi_mask['BGS_ANY'] != 0)
vac_data_sv1 = vac_data_f[sv1_bgs]

## then select by bgs bright
sv1_bgs_bright = (vac_data_sv1 ["SV1_BGS_TARGET"] & sv1_targetmask.bgs_mask['BGS_BRIGHT'] != 0)

print(len(vac_data_sv1[sv1_bgs_bright ]))



55283
